
# DSCI 525 - Web and Cloud Computing
### Group 04: Heidi Ye, Junting He, Kamal Moravej, Tanmay Sharma
#### Date: 16-04-2021
### Repo Link: https://github.com/UBC-MDS/group4-525

Milestone 2: The purpose of this milestone is to move our data to the cloud and get comfortable working there.

### 1. Setup your EC2 instance

- [X] Choose AMI "Ubuntu Server 18.04 LTS (HVM), SSD Volume Type 64-bit (x86)".


- [X] Choose an Instance Type t2.xlarge.


- [X]  Make sure you go with the default VPC & subnet.


- [X]  Get the configuration code from step 7 in the above link and replace "admin-user-name"(remove < > as well) with your AWS IAM username.


- [X] Storage use Root with size 30 GB.


- [X]  Add tag, enter "Owner" under the Key field. In the Value field in the Name row, give your IAM username.


- [X] Select an existing security group Name "DSCI525."


- [X]  Review page looks like this before you launch the instance.


- [X]  In the pop-up, "Select an existing key pair or a new key pair." If you are setting up your cluster for the first time, click on create a new key pair and name it as your "IAM user account". Download the private key and keep it secure. Next time when you set up the EC2 instance, make sure you select "Choose an existing keypair" and choose the one you already made for you.


- [X]  Search for your "IAM user account" under instances to see if it's running. Give it 15 - 20 min as even if it shows running, it will take more time to set up JupyterHub. So please wait...!


- [X]  Check out the "Connect" button to determine how you can connect to the instance. Now you are given the DOOR access to the server, which we mentioned in our first class. :)

![](img/aws_EC.JPG)

### 2. Setup your JupyterHub

- [X] Under description, check for "IPv4 Public IP" and paste the IP address in your browser for your JupyterHub.


- [X] Enter your "IAM user account" and use a strong password & note it down somewhere, as what you enter here will be the admin password.


- [X] In your JupyterHub, go to "Control Panel" --> "admin." Here add other members of your group use their "IAM user account" and make them admins.


- [X] Check if other members can log in to the JupyterHub from their machines by giving them the URL to connect. Step 2.2 is applicable here for other members.

![](img/aws_users.JPG)

### 3. Setup the server

- [X] Login in to the server (instance). The person who spins up the EC2 instance will only have access to the server as he only got the private key. If someone else wants to log in to that instance, you need to get hold of that private key ( Refer 1.10 ).


- [X] Setup a common data folder to download data, and this folder should be accessible by all users in the JupyterHub. Following commands make a folder and make it accessible to everyone. Want to learn more about basic UNIX commands?


> sudo mkdir -p /srv/data/my_shared_data_folder

> sudo chmod 777 /srv/data/my_shared_data_folder/

- [X](OPTIONAL, no bonus points) If you want a sharing notebook environment, then check out this. if you plan to do this, make sure you install the "members" package in your server run sudo apt-get install members."


- [X] Install AWS CLI. More details here.


> NOTE:We are installing this in our EC2 instance, but we can install this anywhere to interact with s3. Say you can install it in your local machine and move data to s3.


```
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
sudo apt install unzip
unzip awscliv2.zip
sudo ./aws/install
```

- [X] Setup your access key and secret. Do it from your AWS console. Make sure you keep your "Access key ID" & secret key somewhere safe.


- [X] Use these credentials to configure AWS CLI (aws configure). More details here. "Default region" and "output format" you can leave empty.


- [X] AWS cli can be used to interact with a lot of services. Check this out. To get a feel, we will use CLI to interact with s3 and wait for step 6.

![](img/aws.JPG)

### 4. Get the data what we wrangled in our first milestone

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

/opt/tljh/user/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
article_id = 14226968  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "/srv/data/my_shared_data_folder/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'allyears.csv.zip',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'id': 26844650,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'size': 2405908113},
 {'is_link_only': False,
  'name': 'individual_years.zip',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77',
  'id': 26863682,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'size': 1896206676},
 {'is_link_only': False,
  'name': 'combined_model_data.csv.zip',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'id': 27515426,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'size': 821308997},
 {'is_link_only': False,
  'name': 'combined_model_data_parti.parquet.zip',
  'supplied_md5': '02f4e3df8d16580a02291de225072689',
  'computed_md5': '02f4e3df8d16580a02

In [4]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [5]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

### 5. Setup your S3 bucket and move data

- [X] Get comfortable with S3 UI. Go from the AWS console.


- [X] Create a bucket there. The name should be mds-s3-xxx. Replace xxx with your "IAM user account".


- [X] All other options leave as it is. (Make sure AWS region is Canada).


- [X] Create your first folder called "output".


- [X] Move the "observed_daily_rainfall_SYD.csv" file from the Milestone1 data folder to your s3 bucket from your local computer. (it's a tiny file, so maybe you can easily use UI to upload).


- [X] Moving the parquet file we downloaded in step 4 to S3 using the cli what we installed in step 3.7. Refer this document and figure out yourself!

![](img/aws_bucket.JPG)

### 6. Wrangle the data in preparation for machine learning

In [7]:
aws_credentials ={"key": "","secret": ""}  
df = pd.read_parquet("s3://mds-s3-student47/combined_model_data_parti.parquet", storage_options=aws_credentials)

In [8]:
syd_lat = -33.86
syd_lon = 151.21
df = df.query("lat_min < @syd_lat < lat_max & lon_min < @syd_lon < lon_max")
df = df.drop(columns = ["lat_min", "lat_max", "lon_min", "lon_max"]).set_index("time")
df

,rain (mm/day),model
time,,
1889-01-01 12:00:00,0.040427,ACCESS-CM2
1889-01-02 12:00:00,0.073777,ACCESS-CM2
1889-01-03 12:00:00,0.232656,ACCESS-CM2
1889-01-04 12:00:00,0.911319,ACCESS-CM2
1889-01-05 12:00:00,0.698013,ACCESS-CM2
...,...,...
2014-12-27 12:00:00,17.444923,TaiESM1
2014-12-28 12:00:00,1.569647,TaiESM1
2014-12-29 12:00:00,1.444630,TaiESM1


In [9]:
df  = df.pivot(columns = "model", values = "rain (mm/day)").resample("1D").mean()
df

model,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MIROC6,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,3.557934e+01,4.268112e+00,1.107466e-03,1.141054e+01,3.322009e-08,2.668800,1.321215,1.515293,...,6.480307e-01,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933
1889-01-02,0.073777,0.303965,4.596520e+00,1.190141e+00,1.015323e-04,4.014984e+00,1.312700e+00,0.946211,2.788724,4.771375,...,1.099883e+00,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381
1889-01-03,0.232656,0.019976,5.927467e+00,1.003845e-09,1.760345e-05,9.660565e+00,9.103720e+00,0.431999,0.003672,4.233980,...,3.931751e+00,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909
1889-01-04,0.911319,13.623777,8.029624e+00,8.225225e-02,1.808932e-01,3.951528e+00,1.317160e+01,0.368693,0.013578,15.252495,...,6.273648e-01,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737
1889-01-05,0.698013,0.021048,2.132686e+00,2.496841e+00,4.708019e-09,2.766362e+00,1.822940e+01,0.339267,0.002468,11.920356,...,1.977652e-02,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-27,0.033748,0.123476,1.451179e+00,3.852845e+01,2.061717e-03,8.179260e-09,1.171263e-02,0.090786,59.895053,5.071783,...,2.916075e-11,4.726998e-13,1.326889e-01,1.827857e+00,6.912632e-03,2.171327e-03,1.620489e+00,2.084252,0.868046,17.444923
2014-12-28,0.094198,2.645496,4.249335e+01,5.833801e-01,5.939502e-09,8.146937e-01,4.938899e-01,0.000000,0.512632,1.578188,...,9.565743e-01,4.609420e-13,1.644482e+00,7.242920e-01,2.836752e-03,1.344768e+01,2.391159e+00,1.644527,0.782258,1.569647
2014-12-29,0.005964,3.041667,2.898325e+00,9.359547e-02,2.000051e-08,2.532205e-01,1.306046e+00,0.000002,37.169669,1.565885,...,7.098663e-01,2.016156e+01,1.506439e+00,1.049481e-01,8.137182e+00,2.547820e+01,1.987695e-12,0.205036,2.140723,1.444630


In [10]:
df_observed = pd.read_csv("s3://mds-s3-student47/observed_daily_rainfall_SYD.csv", storage_options=aws_credentials)
df_observed = df_observed.set_index("time")
df_observed

,rain (mm/day)
time,
1889-01-01,0.006612
1889-01-02,0.090422
1889-01-03,1.401452
1889-01-04,14.869798
1889-01-05,0.467628
...,...
2014-12-27,0.037472
2014-12-28,0.158061
2014-12-29,0.025719


In [11]:
df = df.merge(df_observed, on=df.index).set_index('key_0')
df  = df.rename(columns = {"rain (mm/day)":"Observed"})
df.index.rename("time", inplace=True)
df

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,Observed
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,3.557934e+01,4.268112e+00,1.107466e-03,1.141054e+01,3.322009e-08,2.668800,1.321215,1.515293,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520e+00,1.190141e+00,1.015323e-04,4.014984e+00,1.312700e+00,0.946211,2.788724,4.771375,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467e+00,1.003845e-09,1.760345e-05,9.660565e+00,9.103720e+00,0.431999,0.003672,4.233980,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624e+00,8.225225e-02,1.808932e-01,3.951528e+00,1.317160e+01,0.368693,0.013578,15.252495,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686e+00,2.496841e+00,4.708019e-09,2.766362e+00,1.822940e+01,0.339267,0.002468,11.920356,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-27,0.033748,0.123476,1.451179e+00,3.852845e+01,2.061717e-03,8.179260e-09,1.171263e-02,0.090786,59.895053,5.071783,...,4.726998e-13,1.326889e-01,1.827857e+00,6.912632e-03,2.171327e-03,1.620489e+00,2.084252,0.868046,17.444923,0.037472
2014-12-28,0.094198,2.645496,4.249335e+01,5.833801e-01,5.939502e-09,8.146937e-01,4.938899e-01,0.000000,0.512632,1.578188,...,4.609420e-13,1.644482e+00,7.242920e-01,2.836752e-03,1.344768e+01,2.391159e+00,1.644527,0.782258,1.569647,0.158061
2014-12-29,0.005964,3.041667,2.898325e+00,9.359547e-02,2.000051e-08,2.532205e-01,1.306046e+00,0.000002,37.169669,1.565885,...,2.016156e+01,1.506439e+00,1.049481e-01,8.137182e+00,2.547820e+01,1.987695e-12,0.205036,2.140723,1.444630,0.025719


In [12]:
df.to_csv("s3://mds-s3-student47/output/ml_data_SYD.csv", storage_options=aws_credentials)

![](img/data.JPG)